In [1]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_dir = os.environ['FINN_ROOT'] + "/notebooks/end2end_example/cnn-dizeratie"
model_file = model_dir + "/drowsy-cnn-ready.onnx"

estimates_output_dir = "output_estimates_only"

#Delete previous run results if exist
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

Previous run results deleted!


In [2]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates)

Building dataflow accelerator from /home/stefan/finn/notebooks/end2end_example/cnn-dizeratie/drowsy-cnn-ready.onnx
Intermediate outputs will be generated in /tmp/finn_dev_stefan
Final outputs will be generated in output_estimates_only
Build log is at output_estimates_only/build_dataflow.log
Running step: step_qonnx_to_finn [1/10]
Running step: step_tidy_up [2/10]
Running step: step_streamline [3/10]
Running step: step_convert_to_hw [4/10]
Running step: step_create_dataflow_partition [5/10]
Running step: step_specialize_layers [6/10]
Running step: step_target_fps_parallelization [7/10]
Running step: step_apply_folding_config [8/10]
Running step: step_minimize_bit_width [9/10]
Running step: step_generate_estimate_reports [10/10]
Completed successfully
CPU times: user 1.79 s, sys: 0 ns, total: 1.79 s
Wall time: 1.8 s


0

In [3]:
assert os.path.exists(estimates_output_dir + "/report/estimate_network_performance.json")

In [4]:
! ls {estimates_output_dir}

auto_folding_config.json  report
build_dataflow.log	  template_specialize_layers_config.json
intermediate_models	  time_per_step.json


In [5]:
! ls {estimates_output_dir}/report

estimate_layer_config_alternatives.json  estimate_network_performance.json
estimate_layer_cycles.json		 op_and_param_counts.json
estimate_layer_resources.json


In [6]:
! cat {estimates_output_dir}/report/estimate_network_performance.json

{
  "critical_path_cycles": 140376,
  "max_cycles": 20480,
  "max_cycles_node_name": "StreamingMaxPool_hls_0",
  "estimated_throughput_fps": 4882.8125,
  "estimated_latency_ns": 1403760.0
}

In [7]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

In [8]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_cycles.json")

{'StreamingMaxPool_hls_0': 20480,
 'FMPadding_rtl_0': 4356,
 'ConvolutionInputGenerator_rtl_0': 4358,
 'MVAU_hls_0': 16384,
 'StreamingMaxPool_hls_1': 5120,
 'FMPadding_rtl_1': 1156,
 'ConvolutionInputGenerator_rtl_1': 18574,
 'MVAU_hls_1': 16384,
 'StreamingMaxPool_hls_2': 1280,
 'FMPadding_rtl_2': 324,
 'ConvolutionInputGenerator_rtl_2': 18744,
 'MVAU_hls_2': 16384,
 'StreamingMaxPool_hls_3': 320,
 'MVAU_hls_3': 16384,
 'MVAU_hls_4': 128}

In [9]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_resources.json")

{'StreamingMaxPool_hls_0': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 0,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'FMPadding_rtl_0': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 0,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'ConvolutionInputGenerator_rtl_0': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 364,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MVAU_hls_0': {'BRAM_18K': 8,
  'BRAM_efficiency': 0.0078125,
  'LUT': 3173,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'StreamingMaxPool_hls_1': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 0,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'FMPadding_rtl_1': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 0,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'ConvolutionInputGenerator_rtl_1': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 364,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MVAU_hls_1': {'BRAM_18K': 8,
  'BRAM_efficiency': 0.03125,
  'LUT': 3177,
  'URAM': 0,
 